# Real VS Fake Faces Classification

## Model evaluation

In [21]:
import os

from sklearn.metrics import accuracy_score, classification_report

import utils
import model

## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** path to the config.yml file used for trainning.
- **WEIGHTS:** path to the model weights (.h5 file) to evaluate


In [22]:
CONFIG_YML = "config.yml"
WEIGHTS = "weights/model.better.h5"

In [25]:
config = utils.load_config(CONFIG_YML)
config

{'seed': 21,
 'data': {'directory': 'data/subset',
  'test_dataset': '../data/rd_test_dataset',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [256, 256],
  'batch_size': 20},
 'model': {'weights': 'imagenet',
  'input_shape': [256, 256, 3],
  'classes': 2,
  'data_aug_layer': None,
  'regularizer': 'l2',
  'trainable': True},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.005,
    'epsilon': 0.05,
    'amsgrad': True}},
  'loss': 'binary_crossentropy',
  'metrics': ['categorical_accuracy']},
 'fit': {'epochs': 30,
  'callbacks': {'model_checkpoint': {'filepath': 'model/weights/model.better.h5',
    'save_best_only': True},
   'early_stopping': {'monitor': 'val_loss',
    'patience': 2,
    'restore_best_weights': True},
   'tensor_board': {'log_dir': 'model/weights/logs'}}}}

In [30]:
CLASSES = utils.get_class_names(config)
TEST_FOLDER = config['data']['test_dataset']

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'data/subset'

In [ ]:
cnn_model = model.create_model(weights=WEIGHTS)

print(cnn_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 256, 256, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 256, 256, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                             

## Get predictions from testing dataset

In [28]:
predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


NameError: name 'CLASSES' is not defined

In [ ]:
print(classification_report(y_true=labels, y_pred=predictions))

                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.84      0.95      0.89        44
                             Acura Integra Type R 2001       0.79      0.93      0.85        44
                                   Acura RL Sedan 2012       0.72      0.81      0.76        32
                                   Acura TL Sedan 2012       0.76      0.98      0.86        43
                                  Acura TL Type-S 2008       0.91      0.95      0.93        42
                                  Acura TSX Sedan 2012       0.93      0.70      0.80        40
                              Acura ZDX Hatchback 2012       0.81      0.87      0.84        39
              Aston Martin V8 Vantage Convertible 2012       0.65      0.82      0.73        45
                    Aston Martin V8 Vantage Coupe 2012       0.85      0.71      0.77        41
                  Aston Martin Virage C

In [ ]:
acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Model accuracy {acc:.4f}!")

Your model accuracy is 0.8591!
